# Files and Exception Handling

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atsuyaourt/gis-python-tutorial/blob/main/book/05_python_file.ipynb)

## Creating a Sample File

Before working with files, it's essential to ensure that the files you intend to process actually exist. In this section, you'll learn how to create a sample `tc_track.txt` file programmatically. This file will be used in subsequent examples.

In [3]:
# Create a sample tc_track.txt file
sample_data = """'2009-09-25 18:00',14.9,125.0,35
'2009-09-26 00:00',15.5,123.5,35
'2009-09-26 06:00',15.4,121.8,35
'2009-09-26 12:00',15.5,119.7,35
'2009-09-26 18:00',15.7,118.1,40
'2009-09-27 00:00',15.5,116.4,45
'2009-09-27 06:00',15.2,115.5,55
'2009-09-27 12:00',15.8,114.2,55
'2009-09-27 18:00',15.8,113.4,60
'2009-09-28 00:00',15.8,112.8,65
'2009-09-28 12:00',16.1,111.0,90
'2009-09-28 18:00',16.0,110.2,90
'2009-09-29 00:00',15.5,109.5,90
'2009-09-29 06:00',15.4,108.9,90
'2009-09-29 12:00',15.4,107.8,55
'2009-09-30 00:00',15.3,105.3,55"""

output_file = "tc_track.txt"

try:
    with open(output_file, "w") as file:
        file.write(sample_data)
    print(f"Sample file '{output_file}' has been created successfully.")
except Exception as e:
    print(f"An error occurred while creating the file: {e}")

Sample file 'tc_track.txt' has been created successfully.


In this code, we create a simple text file named `tc_track.txt` containing the timestamp, latitude and longitude of the tropical cyclone center and the corresponding intensity of Ketsana. The file is written in the current working directory.

After running this script, the `tc_track.txt` file will be available for use in the following examples. If any issues occur during the file creation process, the script will handle them and print an error message.

## Working with Files

In geospatial programming, you often need to read from or write to files. Python provides built-in functions to handle these tasks. Let's start by reading from a text file containing coordinates and writing the results to a new file.

In [4]:
# Example of reading coordinates from a file and writing to another file
input_file = "tc_track.txt"
output_file = "output_coordinates.txt"

try:
    with open(input_file, "r") as infile:
        coordinates = infile.readlines()

    with open(output_file, "w") as outfile:
        for line in coordinates:
            ts, lat, lon, vmax = line.strip().split(",")
            outfile.write(f"Latitude: {lat}, Longitude: {lon}\n")

    print(f"Coordinates have been written to {output_file}")
except FileNotFoundError:
    print(f"Error: The file {input_file} was not found.")

Coordinates have been written to output_coordinates.txt


## Exception Handling

Exception handling allows you to handle errors that occur during the execution of your program. This is especially important in geospatial programming, where you may encounter issues such as missing files, corrupt data, or invalid input.

Let's explore how to handle different types of exceptions using `try`, `except`, and `finally`.

In [5]:
# Example of exception handling when parsing coordinates
def parse_coordinates(line):
    try:
        _, lat, lon, _ = line.strip().split(",")
        lat = float(lat)
        lon = float(lon)
        return lat, lon
    except ValueError as e:
        print(f"Error: {e}. Could not parse line: {line.strip()}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


# Example usage
line = "invalid data"
coordinates = parse_coordinates(line)
if coordinates:
    print(f"Parsed coordinates: {coordinates}")

Error: not enough values to unpack (expected 4, got 1). Could not parse line: invalid data


## Combining File Handling and Exception Handling

You can combine file handling and exception handling to create robust geospatial applications. This allows you to ensure that files are properly handled even if errors occur during the process.

In [6]:
# Example of robust file handling with exceptions
def process_geospatial_file(input_file):
    try:
        with open(input_file, "r") as infile:
            for line in infile:
                coordinates = parse_coordinates(line)
                if coordinates:
                    print(f"Processed coordinates: {coordinates}")
    except FileNotFoundError:
        print(f"Error: The file {input_file} was not found.")
    except Exception as e:
        print(f"An unexpected error occurred while processing the file: {e}")
    finally:
        print(f"Finished processing {input_file}")


# Example usage
process_geospatial_file("tc_track.txt")

Processed coordinates: (14.9, 125.0)
Processed coordinates: (15.5, 123.5)
Processed coordinates: (15.4, 121.8)
Processed coordinates: (15.5, 119.7)
Processed coordinates: (15.7, 118.1)
Processed coordinates: (15.5, 116.4)
Processed coordinates: (15.2, 115.5)
Processed coordinates: (15.8, 114.2)
Processed coordinates: (15.8, 113.4)
Processed coordinates: (15.8, 112.8)
Processed coordinates: (16.1, 111.0)
Processed coordinates: (16.0, 110.2)
Processed coordinates: (15.5, 109.5)
Processed coordinates: (15.4, 108.9)
Processed coordinates: (15.4, 107.8)
Processed coordinates: (15.3, 105.3)
Finished processing tc_track.txt


## Reading GeoJSON Data from a URL

In geospatial programming, it's common to access datasets hosted online. For instance, we may need to read GeoJSON data directly from a URL. Python's requests library makes it easy to retrieve such data. In this example, we'll read a GeoJSON file containing U.S. cities and handle any exceptions that may arise.

First, ensure you have the requests library installed. Uncomment and run the following command if you haven't already installed it.

In [ ]:
# !pip install requests

Import the required libraries and read the GeoJSON data from the URL.

In [7]:
import requests

In this case, we will read the GeoJSON data for Philippine regions hosted at https://github.com/faeldon/philippines-json-maps, and process it to extract the names and land area of the regions.

In [10]:
url = (
    "https://raw.githubusercontent.com/faeldon/philippines-json-maps/refs/heads/master/2023/geojson/country/lowres/country.0.001.json"
)

Let's define a function to read the GeoJSON data from the URL and extract the city names and coordinates. We'll also handle any exceptions that may occur during the process.

In [11]:
def fetch_geojson(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an exception for HTTP errors
        geojson_data = response.json()  # Parse the JSON response
        return geojson_data
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Error connecting to the server: {conn_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    return None


# Fetch and print a summary of the data
geojson_data = fetch_geojson(url)

Uncomment and run the following code to display the data read from the URL.

In [13]:
# geojson_data

The output above is lengthy, so we'll only display the first few regions. You can modify the code to display more cities if needed.

In [23]:
if geojson_data:
    features = geojson_data.get("features", [])
    print(f"Number of regions: {len(features)}")

    # Extract region names and area
    for feature in features[:5]:  # Display first 5 cities
        region_name = feature["properties"].get("adm1_en")
        area_km2 = feature["properties"].get("area_km2")
        print(f"Name: {region_name}, Area: {area_km2} km^2")

Number of regions: 17
Name: Region I (Ilocos Region), Area: 12319 km^2
Name: Region II (Cagayan Valley), Area: 26379 km^2
Name: Region III (Central Luzon), Area: 21317 km^2
Name: Region IV-A (CALABARZON), Area: 15844 km^2
Name: Region V (Bicol Region), Area: 17326 km^2


## Exercises

1. Create a function that reads a file containing a list of city names and their coordinates. The function should handle exceptions if the file is missing or if a line in the file is not properly formatted.
2. Write a function that writes a list of coordinates to a file. Include exception handling to ensure that the file is properly closed even if an error occurs during writing.
3. Create a robust geospatial data processing function that reads data from a file, processes it, and writes the results to another file. Ensure that all potential errors are handled appropriately.